In [62]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import sys
import re
from operator import itemgetter

from nltk.stem.porter import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, accuracy_score, mean_squared_error


In [2]:

def _stem(doc, p_stemmer, en_stop, return_tokens):
    tokens = word_tokenize(doc.lower())
    stopped_tokens = filter(lambda token: token not in en_stop, tokens)
    stemmed_tokens = map(lambda token: p_stemmer.stem(token), stopped_tokens)
    if not return_tokens:
        return ' '.join(stemmed_tokens)
    return list(stemmed_tokens)

def getStemmedDocuments(docs, return_tokens=True):
    """
        Args:
            docs: str/list(str): document or list of documents that need to be processed
            return_tokens: bool: return a re-joined string or tokens
        Returns:
            str/list(str): processed document or list of processed documents
        Example: 
            new_text = "It is important to by very pythonly while you are pythoning with python. \
                All pythoners have pythoned poorly at least once."
            print(getStemmedDocuments(new_text))
        Reference: https://pythonprogramming.net/stemming-nltk-tutorial/
    """
    en_stop = set(stopwords.words('english'))
    ps = PorterStemmer()
    if isinstance(docs, list):
        output_docs = []
        for item in docs:
            output_docs.append(_stem(item, ps, en_stop, return_tokens))
        return output_docs
    else:
        return _stem(docs, ps, en_stop, return_tokens)

In [3]:
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [4]:
LaplaceSmoothing = 1
BASE_DIR = "../"
train_path = os.path.join(BASE_DIR, 'data', 'reviews_Digital_Music_5.json', 'Music_Review_train.json')
test_path  = os.path.join(BASE_DIR, 'data', 'reviews_Digital_Music_5.json', 'Music_Review_test.json')

In [5]:
def load_data(filename):
    return pd.read_json(filename, lines=True)

In [6]:
def clean_data(line):
    line = line.strip().lower()
    line = re.sub(r'[^\w\s]','',line)
    line = re.sub('\r?\n',' ',line)
    return line

def preprocessing(df, stemming):
    ColumnsToDrop = ['reviewerID', 'asin', 'reviewerName', 'unixReviewTime', 'reviewTime']
    df = df.drop(ColumnsToDrop, axis=1)
    
    df['reviewText'].apply(lambda x: clean_data(x))
    df['summary'].apply(lambda x: clean_data(x))
    
    if stemming:
        df['reviewText'].apply(lambda x: getStemmedDocuments(x))
        df['summary'].apply(lambda x: getStemmedDocuments(x))
        # for i in range(df.shape[0]):
        #     print(i)
        #     df.iloc[i,1] = getStemmedDocuments(df.iloc[i,1], False)
        #     df.iloc[i,3] = getStemmedDocuments(df.iloc[i,3], False)    
    return df

In [7]:
def Vocab_generation(df1, df2):
    VocabSize = 0
    Vocab = dict()
    
    min_val = min(df2)
    max_val = max(df2)
    
    VocabClassSize = [0]*(max_val - min_val + 1)
    ExampleSize    = [0]*(max_val - min_val + 1)    
        
    for i,review in enumerate(df1):
        for word in review.split():
            if word not in Vocab:
                Vocab[word] = 1
                VocabSize  += 1

    VocabClass = []
    for i in range(max_val - min_val + 1):
        d = Vocab.copy()
        VocabClass.append(d)
    
    for i,review in enumerate(df1):
        ExampleSize[df2[i]-min_val] += 1
        for word in review.split():
            VocabClassSize[df2[i]-min_val] += 1
            VocabClass[df2[i]-min_val][word] += 1
    
    return Vocab, VocabClass, VocabSize, VocabClassSize, ExampleSize

In [8]:
part_d = False

train_data = preprocessing(load_data(train_path), part_d)
test_data  = preprocessing(load_data(test_path), part_d)

X_train = train_data['reviewText'].copy()
Y_train = train_data['overall'].copy()

X_test = test_data['reviewText'].copy()
Y_test = test_data['overall'].copy()

In [9]:
Vocab, VocabClass, VocabSize, VocabClassSize, phi = Vocab_generation(X_train, Y_train)

In [10]:
v1, v2, v3, v4, v5 = Vocab, VocabClass, VocabSize, VocabClassSize, phi
print(phi)
print(VocabClassSize)
# print(VocabClass)

[2529, 2638, 5634, 13267, 25932]
[328626, 470556, 1183227, 2902715, 4954673]


In [11]:
# for i in range(len(VocabClass)):
#     for x in VocabClass[i]:
#         if(VocabClass[i][x] <= 0):
#             print("ERROR",i,x)
#         else:
#             np.log(float(VocabClass[i][x])/float(VocabClassSize[i] + VocabSize))
# print("DONE")

DONE


In [12]:
for i in range(len(VocabClass)):
    for x in VocabClass[i]:
        if(VocabClass[i][x] <= 0):
            print("ERROR FOR CLASS:",i)
        else:
            VocabClass[i][x] = np.log(float(VocabClass[i][x])/float(VocabClassSize[i] + VocabSize))
    
for i in range(len(VocabClass)):
    phi[i] = np.log(float(phi[i])/float(len(X_train)))

In [55]:
def predict(X,Y,phi,VocabClass,VocabClassSize):
    Y_pred = [0]*(Y.shape[0])
    count = 0
    minval = min(Y)
    
    for i,review in enumerate(X):
        class_prob = [0.0]*len(VocabClass)

        for word in review.split():
            for j in range(len(VocabClass)):
                if word in VocabClass[j]:
                    class_prob[j] += VocabClass[j][word]
                else:
                    class_prob[j] += (float(LaplaceSmoothing)/float(VocabSize))
                
        for j in range(len(VocabClass)):
            class_prob[j] += phi[j]
            
        # class_label = max(enumerate(class_prob, key=lambda x: x[1]))[0] + min(Y)
        class_label, element = max(enumerate(class_prob), key=itemgetter(1))
        class_label += minval
        
        Y_pred[i]   = class_label
        
        if class_label == Y[i]:
            count+=1
        
        # print(class_prob, element, "Predicted:", class_label, "Correct: ", Y[i])
        # print(i)
        
    return count, Y_pred

In [20]:
def f1_score(A,B):
    conf_mat = np.zeros((5,5));
    for i in range(B.shape[0]):
        conf_mat[int(B[i]-1)][int(A[i]-1)] = conf_mat[int(B[i]-1)][int(A[i]-1)] + 1;
    
    p1 = np.zeros(5);
    r1 = np.zeros(5);
    f1 = np.zeros(5);

    for i in range(5):
        p1[i] = conf_mat[i,i]/(np.sum(conf_mat, axis = 0)[i])
        r1[i] = conf_mat[i,i]/(np.sum(conf_mat, axis = 1)[i])
        f1[i] = (2*p1[i]*r1[i])/(p1[i]+r1[i]);

    return round(np.average(f1),5);

correct, Y_pred_train = predict(X_train, Y_train, phi, VocabClass, VocabClassSize)
print("Train Set Accuracy = {}%".format(round(100*float(correct)/float(len(X_train)),2)))
# print("Macro F1 score =", f1_score(Y_pred_train, Y_train))

correct, Y_pred_test = predict(X_test, Y_test, phi, VocabClass, VocabClassSize)
print("Test Set Accuracy  = {}%".format(round(100*float(correct)/float(len(X_test)),2)))
# print("Macro F1 score =", f1_score(Y_pred_test, Y_test))

Train Set Accuracy = 72.086%
Test Set Accuracy = 66.564%


In [26]:
# Part B
correct, Y_pred_test = predict(X_test, Y_test, phi, VocabClass, VocabClassSize)
print("Test Set Accuracy = {}%".format(round(100*float(correct)/float(len(X_test)),2)))

Test Set Accuracy = 66.56%


In [39]:
# Random Guessing
def random_guessing():
    Y_pred = np.zeros(len(Y_test))
    count = 0
    for i in range (Y_test.shape[0]):
        class_label = np.random.randint(1,6)
        Y_pred[i]   = class_label
        if(class_label == Y_test[i]):
            count+=1
    return count, Y_pred

correct, Y_pred = random_guessing()
print("Random Prediction Accuracy (Test Set) = {}%".format(round(100*float(correct)/float(len(X_test)),2)))

# Majority Prediction
def majority_prediction():
    Y_pred = np.zeros(len(Y_test))
    count = 0
    class_label, element = max(enumerate(phi), key=itemgetter(1))
    class_label += 1
    
    for i in range (Y_test.shape[0]):        
        Y_pred[i]   = class_label
        if(class_label == Y_test[i]):
            count+=1
    return count, Y_pred

correct, Y_pred = majority_prediction()
print("Majority Prediction Accuracy (Test Set) = {}%".format(round(100*float(correct)/float(len(X_test)),2)))

Random Prediction Accuracy (Test Set) = 20.17%
Majority Prediction Accuracy (Test Set) = 66.09%


In [42]:
# Part C
correct, Y_pred_test = predict(X_test, Y_test, phi, VocabClass, VocabClassSize)

In [45]:
confusion_matrix = np.zeros((5,5))
for i in range(len(Y_test)):
    confusion_matrix[Y_test[i]-1][Y_pred_test[i]-1] += 1
print(confusion_matrix.astype(int))

[[   2    0    0   21  205]
 [   0    0    0   63  263]
 [   2    0    1  231  852]
 [   3    0    0  280 2825]
 [  14    0    0  202 9036]]


In [46]:
# Part D
part_d = True
train_data = preprocessing(load_data(train_path), part_d)
test_data  = preprocessing(load_data(test_path), part_d)

X_train = train_data['reviewText'].copy()
Y_train = train_data['overall'].copy()

X_test = test_data['reviewText'].copy()
Y_test = test_data['overall'].copy()

In [56]:
Vocab, VocabClass, VocabSize, VocabClassSize, phi = Vocab_generation(X_train, Y_train)

for i in range(len(VocabClass)):
    for x in VocabClass[i]:
        if(VocabClass[i][x] <= 0):
            print("ERROR FOR CLASS:",i)
        else:
            VocabClass[i][x] = np.log(float(VocabClass[i][x])/float(VocabClassSize[i] + VocabSize))

for i in range(len(VocabClass)):
    phi[i] = np.log(float(phi[i])/float(len(X_train)))
    
correct, Y_pred_train = predict(X_train, Y_train, phi, VocabClass, VocabClassSize)
print("Train Set Accuracy (stemmed data) = {}%".format(round(100*float(correct)/float(len(X_train)),2)))

correct, Y_pred_test = predict(X_test, Y_test, phi, VocabClass, VocabClassSize)
print("Test Set Accuracy (stemmed data)  = {}%".format(round(100*float(correct)/float(len(X_test)),2)))

Train Set Accuracy (stemmed data) = 72.09%
Test Set Accuracy (stemmed data)  = 66.56%


In [60]:
# duffelzaar
# Part E
vectorizer = TfidfVectorizer(preprocessor=None,
                            tokenizer = word_tokenize,
                            analyzer='word',
                            stop_words=None,
                            strip_accents=None, 
                            lowercase=True,
                            ngram_range=(1,3), 
                            min_df=0.0001, 
                            max_df=0.9,
                            binary=False,
                            norm='l2',
                            use_idf=1,
                            smooth_idf=1, 
                            sublinear_tf=1)

X_train = vectorizer.fit_transform(X_train)
X_test  = vectorizer.transform(X_test)

mnb = MultinomialNB()
mnb.fit(X_train,Y_train)

pred_mnb = mnb.predict(X_test)
print("Feature Engineering Score:",round(accuracy_score(Y_test,pred_mnb),3));

Feature Engineering Score: 0.661


In [70]:
# Part F
# correct, Y_pred_test = predict(X_test, Y_test, phi, VocabClass, VocabClassSize)
print("Test Set Accuracy = {}%".format(round(100*float(correct)/14000.0),2))
print("Macro F1 score = ".format(f1_score(Y_pred_test, Y_test)))

Test Set Accuracy = 67%
Macro F1 score = 


/tmp/ipykernel_32215/2420841967.py:11: RuntimeWarning: invalid value encountered in double_scalars
  p1[i] = conf_mat[i,i]/(np.sum(conf_mat, axis = 0)[i])
